## Activation maximization

---

A quick demo of activation maximization with [FlashTorch 🔦](https://github.com/MisaOgura/flashtorch), using the pre-trained VGG16 model.


❗This notebook is for those who are using this notebook in **Google Colab**.

If you aren't on Google Colab already, please head to the Colab version of this notebook **[here](https://colab.research.google.com/github/MisaOgura/flashtorch/blob/master/examples/activation_maximization_colab.ipynb)** to execute.

---

[Activation maximization](https://pdfs.semanticscholar.org/65d9/94fb778a8d9e0f632659fb33a082949a50d3.pdf) is one form of feature visualization that allows us to visualize what CNN filters are "looking for", by applying each filter to an input image and updating the input image so as to maximize the activation of the filter of interest (i.e. treating it as a gradient ascent task with filter activation values as the loss).

The optimization and visualization is available via `flashtorch.activmax.GradientAscent`. The implementation is inspired by [this demo](https://blog.keras.io/category/demo.html) by Francois Chollet.

### 0. Set up

In [ ]:
# Install flashtorch if you don't have it

# !pip install flashtorch

In [ ]:
import torchvision.models as models

from flashtorch.activmax import GradientAscent

### 1. Load a pre-trained Model

In [ ]:
model = models.vgg16(pretrained=True)

# Print layers and corresponding indicies

list(model.features.named_children())

### 2. Specify layers and filters

In [ ]:
conv1_2 = model.features[2]
conv1_2_filters = [17, 33, 34, 57]

conv2_1 = model.features[5]
conv2_1_filters = [27, 40, 68, 73]

conv3_1 = model.features[10]
conv3_1_filters = [31, 61, 147, 182]

conv4_1 = model.features[17]
conv4_1_filters = [238, 251, 338, 495]

conv5_1 = model.features[24]
conv5_1_filters = [45, 271, 363, 409]

### 3. Optimize and visualize filters

Creating an instance of `GradientAscent` class with the model _without fully-connected layers_ allows us to use flexible input image sizes.

In [ ]:
g_ascent = GradientAscent(model.features)

By calling the `visualize` method and passing in the layer and filter indeciies defined above, it performs optimization and visualization.

This is perhaps the most common way to use the `GradientAscent` class, but there are other APIs available according to your use cases (see section 4).

In [ ]:
g_ascent.visualize(conv1_2, conv1_2_filters, title='conv1_2');
g_ascent.visualize(conv2_1, conv2_1_filters, title='conv2_1');
g_ascent.visualize(conv3_1, conv3_1_filters, title='conv3_1');
g_ascent.visualize(conv4_1, conv4_1_filters, title='conv4_1');
g_ascent.visualize(conv5_1, conv5_1_filters, title='conv5_1');

### 4. Other ways to use `GradientAscent`

#### 4-1. `GradientAscent.visualize`: randomly select filters

If you have a convolutional layer you want to vizualise, but you don't know which filters to choose, you can just pass in the layer to `visualize` without `filter_idxs`. It will randomly choose filters. You can adjust the number of filters chosen by passing `num_subplots` (default=4).

In [ ]:
g_ascent.visualize(conv5_1, title='Randomly selected filters from conv5_1');

#### 4-2. `GradientAscent.visualize`: plot one filter

If you just want to visualize one filter, you can do so by specifying the filter index as an integer, not a list.

In [ ]:
g_ascent.visualize(conv5_1, 3, title='conv5_1 filter 3');

#### 4-3. `GradientAscent.visualize`: return image tensor

If you want to grab the optimized image data, set `return_output` to `True`.

In [ ]:
output = g_ascent.visualize(conv5_1, 3, title='conv5_1 filter 3', return_output=True);

print('num_iter:', len(output))
print('optimized image:', output[-1].shape)

#### 4-4. `GradientAscent.optimize`: perform optimization only (no visualization)

If no visualization is needed, or if you want to futher customize visualization, you can call the `optimize` method directly.

In [ ]:
output = g_ascent.optimize(conv5_1, 3)

print('num_iter:', len(output))
print('optimized image:', output[-1].shape)